In [42]:
# Import all the necessary modules
import os
import sys
import os, sys
# from .../research/notebooks -> go up two levels to repo root
repo_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
import pandas_datareader as pdr
import math
import datetime
from datetime import datetime, timezone
import itertools
import ast
import yfinance as yf
import seaborn as sn
from IPython.display import display, HTML
from strategy_signal.trend_following_signal import (
    apply_jupyter_fullscreen_css, get_trend_donchian_signal_for_portfolio_with_rolling_r_sqr_vol_of_vol
)
from portfolio.strategy_performance import (calculate_sharpe_ratio, calculate_calmar_ratio, calculate_CAGR, calculate_risk_and_performance_metrics,
                                          calculate_compounded_cumulative_returns, estimate_fee_per_trade, rolling_sharpe_ratio)
from utils import coinbase_utils as cn
from portfolio import strategy_performance as perf
from sizing import position_sizing_binary_utils as size_bin
from sizing import position_sizing_continuous_utils as size_cont
from strategy_signal import trend_following_signal as tf
%matplotlib inline

In [4]:
import importlib
importlib.reload(cn)
importlib.reload(perf)
importlib.reload(tf)
importlib.reload(size_bin)
importlib.reload(size_cont)

<module 'sizing.position_sizing_continuous_utils' from '/Users/adheerchauhan/Documents/git/trend_following/sizing/position_sizing_continuous_utils.py'>

In [6]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns',None)
apply_jupyter_fullscreen_css()

## Helper Functions

In [9]:
from collections import OrderedDict

def print_strategy_params():
    """
    Pretty-print the strategy’s configuration values, with a blank line
    separating each logical section.
    """

    # ---- Define sections (title is just for dev readability) --------------
    sections = [
        ("Dates & universe", OrderedDict([
            ("start_date",      start_date),
            ("end_date",        end_date),
            ("warm_up_days",    WARMUP_DAYS),
            ("ticker_list",     ticker_list),
        ])),

        ("Moving-average / trend", OrderedDict([
            ("fast_mavg",                  fast_mavg),
            ("slow_mavg",                  slow_mavg),
            ("mavg_stepsize",              mavg_stepsize),
            ("mavg_z_score_window",        mavg_z_score_window),
            ("moving_avg_type",            moving_avg_type),
            ("ma_crossover_signal_weight", ma_crossover_signal_weight),
        ])),

        ("Donchian channel", OrderedDict([
            ("entry_rolling_donchian_window", entry_rolling_donchian_window),
            ("exit_rolling_donchian_window", exit_rolling_donchian_window),
            ("use_donchian_exit_gate", use_donchian_exit_gate),
            ("donchian_signal_weight",  donchian_signal_weight),
        ])),

        ("Volatility & risk", OrderedDict([
            ("volatility_window",            volatility_window),
            ("annualized_target_volatility", annualized_target_volatility),
            ("rolling_cov_window",           rolling_cov_window),
            ("rolling_atr_window",           rolling_atr_window),
            ("atr_multiplier",               atr_multiplier),
            ("log_std_window",               log_std_window),
            ("coef_of_variation_window",     coef_of_variation_window),
            ("vol_of_vol_z_score_window",    vol_of_vol_z_score_window),
            ("vol_of_vol_p_min",             vol_of_vol_p_min),
            ("r2_strong_threshold",          r2_strong_threshold)
        ])),

        ("Signal gating / quality", OrderedDict([
            ("lower_r_sqr_limit",             lower_r_sqr_limit),
            ("upper_r_sqr_limit",             upper_r_sqr_limit),
            ("rolling_r2_window",             rolling_r2_window),
            ("r2_smooth_window",              r2_smooth_window),
            ("r2_confirm_days",               r2_confirm_days),
            ("rolling_sharpe_window",         rolling_sharpe_window),
            ("use_activation",                use_activation),
            ("tanh_activation_constant_dict", tanh_activation_constant_dict),
            ("weighted_signal_ewm_window",    weighted_signal_ewm_window)
        ])),

        ("Trading toggles & thresholds", OrderedDict([
            ("long_only",                  long_only),
            ("use_coinbase_data",          use_coinbase_data),
            ("use_saved_files",            use_saved_files),
            ("saved_file_end_date",        saved_file_end_date),
            ("use_specific_start_date",    use_specific_start_date),
            ("signal_start_date",          signal_start_date),
            ("price_or_returns_calc",      price_or_returns_calc),
            ("notional_threshold_pct",     notional_threshold_pct),
            ("cooldown_counter_threshold", cooldown_counter_threshold),
            ("warmup_days",                WARMUP_DAYS)
        ])),

        ("Capital & execution", OrderedDict([
            ("initial_capital",        initial_capital),
            ("cash_buffer_percentage", cash_buffer_percentage),
            ("transaction_cost_est",   transaction_cost_est),
            ("passive_trade_rate",     passive_trade_rate),
            ("annual_trading_days",    annual_trading_days),
        ])),
    ]

    # ---- Compute width for neat alignment ---------------------------------
    longest_key = max(len(k) for _, sec in sections for k in sec)

    print("\nStrategy Parameters\n" + "-" * (longest_key + 30))
    for _, sec in sections:
        for k, v in sec.items():
            print(f"{k:<{longest_key}} : {v}")
        print()  # blank line between sections
    print("-" * (longest_key + 30) + "\n")

# ---------------------------------------------------------------------------
# Example usage (uncomment after your own parameter definitions are in scope)
# ---------------------------------------------------------------------------
# if __name__ == "__main__":
#     print_strategy_params()

In [11]:
def plot_signal_performance(df_1, df_2, ticker):

    fig = plt.figure(figsize=(20,12))
    layout = (2,2)
    signal_ax = plt.subplot2grid(layout, (0,0))
    price_ax = signal_ax.twinx()
    equity_curve_ax = plt.subplot2grid(layout, (0,1))
    sharpe_ax = plt.subplot2grid(layout, (1,0))
    portfolio_value_ax = plt.subplot2grid(layout, (1,1))

    _ = signal_ax.plot(df_1.index, df_1[f'{ticker}_final_signal'], label='Orig Signal', alpha=0.9)
    _ = signal_ax.plot(df_2.index, df_2[f'{ticker}_final_signal'], label='New Signal', alpha=0.9)
    _ = price_ax.plot(df_1.index, df_2[f'{ticker}_open'], label='Price', alpha=0.7, linestyle='--', color='magenta')
    _ = signal_ax.set_title(f'Orignal Signal vs New Signal')
    _ = signal_ax.set_ylabel('Signal')
    _ = signal_ax.set_xlabel('Date')
    _ = signal_ax.legend(loc='upper left')
    _ = signal_ax.grid()

    _ = equity_curve_ax.plot(df_1.index, df_1[f'equity_curve'], label='Orig Signal', alpha=0.9)
    _ = equity_curve_ax.plot(df_2.index, df_2[f'equity_curve'], label='New Signal', alpha=0.9)
    _ = equity_curve_ax.set_title(f'Equity Curve')
    _ = equity_curve_ax.set_ylabel('Equity Curve')
    _ = equity_curve_ax.set_xlabel('Date')
    _ = equity_curve_ax.legend(loc='upper left')
    _ = equity_curve_ax.grid()

    _ = sharpe_ax.plot(df_1.index, df_1[f'portfolio_rolling_sharpe_50'], label='Orig Signal', alpha=0.9)
    _ = sharpe_ax.plot(df_2.index, df_2[f'portfolio_rolling_sharpe_50'], label='New Signal', alpha=0.9)
    _ = sharpe_ax.set_title(f'Rolling Sharpe')
    _ = sharpe_ax.set_ylabel(f'Rolling Sharpe')
    _ = sharpe_ax.set_xlabel('Date')
    _ = sharpe_ax.legend(loc='upper left')
    _ = sharpe_ax.grid()

    _ = portfolio_value_ax.plot(df_1.index, df_1[f'total_portfolio_value'], label='Orig Signal', alpha=0.9)
    _ = portfolio_value_ax.plot(df_2.index, df_2[f'total_portfolio_value'], label='New Signal', alpha=0.9)
    _ = portfolio_value_ax.set_title(f'Total Portfolio Value')
    _ = portfolio_value_ax.set_ylabel('Portfolio Value')
    _ = portfolio_value_ax.set_xlabel('Date')
    _ = portfolio_value_ax.legend(loc='upper left')
    _ = portfolio_value_ax.grid()

    plt.tight_layout()

    return

# Walk Forward Analysis

In [34]:
import itertools
import time
import numpy as np
import pandas as pd

# ---------- Small helpers ----------
def zscore_in_fold(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    m, sd = s.mean(), s.std(ddof=0)
    if np.isfinite(sd) and sd > 0:
        return (s - m) / sd
    return pd.Series(0.0, index=s.index)

def composite_is_score(df_block: pd.DataFrame) -> pd.Series:
    # expects columns: annualized_sharpe_ratio, max_drawdown, trade_count
    z_turn = zscore_in_fold(df_block["trade_count"].fillna(df_block["trade_count"].median()))
    z_mdd  = zscore_in_fold(df_block["max_drawdown"].abs().fillna(df_block["max_drawdown"].abs().median()))
    score  = pd.to_numeric(df_block["annualized_sharpe_ratio"], errors="coerce") \
             - 0.25*z_turn - 0.15*z_mdd
    return score

def make_grid(fixed_params: dict, sweep_params: dict):
    """Yield parameter dicts: fixed merged with each sweep point."""
    keys, values = zip(*sweep_params.items()) if sweep_params else ([], [])
    for prod in (itertools.product(*values) if values else [()]):
        update = dict(zip(keys, prod))
        cfg = fixed_params.copy()
        cfg.update(update)
        yield cfg

def _grid_size(sweep_params: dict) -> int:
    if not sweep_params:
        return 1
    n = 1
    for v in sweep_params.values():
        n *= len(v)
    return n

def _fmt_params(p: dict) -> str:
    keys = [
        'annualized_target_volatility',
        'stop_loss_strategy',
        'atr_multiplier',
        'rolling_atr_window',
        'highest_high_window',
        'cooldown_counter_threshold',
    ]
    return ", ".join(f"{k}={p[k]}" for k in keys if k in p)

def _clean_params(p: dict) -> dict:
    """Remove fold-controlled keys so we can set them explicitly per fold."""
    drop = {"signal_start_date", "use_specific_start_date"}
    return {k: v for k, v in p.items() if k not in drop}

# ---------- Core WFA runner ----------
def run_wfa(
    start_date: str,
    end_date: str,
    ticker_list,
    *,
    is_months=18,
    os_months=3,
    step_equals_os=True,     # non-overlapping for speed
    warmup_days=300,
    min_trades_is=6,         # guardrail
    fixed_params: dict,
    sweep_params: dict,
    tf_fn,                   # your strategy function
    perf_fn,                 # your risk/perf metrics function (portfolio-level)
    include_ticker_metrics=True
):
    """
    Returns df_performance with IS and OS rows.
    Adds: is_score on IS rows; os_score (Sharpe) on OS rows; is_promoted flag on OS row.
    Uses YAML parameter names throughout.
    """
    start_ts = time.perf_counter()
    start_date = pd.Timestamp(start_date).date()
    end_date   = pd.Timestamp(end_date).date()

    IS_LEN = pd.DateOffset(months=is_months)
    OS_LEN = pd.DateOffset(months=os_months)
    STEP   = (OS_LEN if step_equals_os else pd.DateOffset(months=1))

    grid_n = _grid_size(sweep_params)
    print(f"[WFA] init: IS={is_months}m, OS={os_months}m, STEP={'OS_LEN' if step_equals_os else '1m'}, "
          f"warmup={warmup_days}d, tickers={len(ticker_list)}, grid_size={grid_n}", flush=True)

    perf_cols = [
        'sampling_category','start_date','end_date',
        'annualized_return','annualized_sharpe_ratio','calmar_ratio',
        'annualized_std_dev','max_drawdown','max_drawdown_duration',
        'hit_rate','t_statistic','p_value','trade_count',
        # params (YAML names)
        'annualized_target_volatility','stop_loss_strategy','atr_multiplier','rolling_atr_window',
        'highest_high_window','cooldown_counter_threshold',
        # scores
        'is_score','os_score','is_promoted'
    ]
    if include_ticker_metrics:
        ticker_perf_cols = ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']
        perf_cols += [f'{t}_{c}' for t in ticker_list for c in ticker_perf_cols]

    df_performance = pd.DataFrame(columns=perf_cols)

    fold_idx = 0
    start_is = start_date
    while True:
        end_is = (start_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_os = (end_is + pd.Timedelta(days=1))
        end_os = (start_os + OS_LEN - pd.Timedelta(days=1)).date()

        if end_os > end_date - pd.Timedelta(days=1):
            print("[WFA] done: OS end exceeds end_date; exiting loop.", flush=True)
            break

        fold_idx += 1
        print(f"\n[WFA] Fold {fold_idx}: IS {start_is} → {end_is} | OS {start_os} → {end_os} | warmup={warmup_days}d", flush=True)

        # ---- 1) Evaluate all sweep params in-sample ----
        is_rows = []
        print(f"[WFA] Fold {fold_idx}: evaluating {grid_n} config(s) in-sample…", flush=True)
        t0 = time.perf_counter()
        for j, p in enumerate(make_grid(fixed_params, sweep_params), start=1):
            p_is = _clean_params(p)
            print(f"[WFA] Fold {fold_idx} | IS cfg {j}/{grid_n}: {_fmt_params(p_is)}", flush=True)
            # 1a) In-sample run (pass fold-local signal_start_date)
            df_is = tf_fn(
                start_date=start_is - pd.Timedelta(days=warmup_days),
                end_date=end_is, ticker_list=ticker_list,
                **p_is,
                use_specific_start_date=True,
                signal_start_date=start_is,
            )
            df_is = df_is[df_is.index >= pd.Timestamp(start_is).date()]

            df_is = perf.calculate_asset_level_returns(df_is, end_is, ticker_list)

            # 1b) Portfolio metrics
            is_port = perf_fn(
                df_is,
                strategy_daily_return_col='portfolio_daily_pct_returns',
                strategy_trade_count_col='count_of_positions',
                include_transaction_costs_and_fees=False,
                passive_trade_rate=0.05, annual_trading_days=365,
                transaction_cost_est=0.001
            )
            print(f"[WFA] Fold {fold_idx} | IS cfg {j}/{grid_n} METRICS: "
                  f"Sharpe={is_port.get('annualized_sharpe_ratio',np.nan):.3f}, "
                  f"MDD={is_port.get('max_drawdown',np.nan):.3%}, "
                  f"trades={int(is_port.get('trade_count',0))}", flush=True)

            row = {
                'sampling_category':'in_sample',
                'start_date': start_is,
                'end_date': end_is,
                'is_score': np.nan, 'os_score': np.nan, 'is_promoted': False,
                **{k: p_is.get(k, np.nan) for k in [
                    'annualized_target_volatility','stop_loss_strategy','atr_multiplier',
                    'rolling_atr_window','highest_high_window','cooldown_counter_threshold'
                ]},
                **is_port
            }

            if include_ticker_metrics:
                for t in ticker_list:
                    tmet = perf.calculate_risk_and_performance_metrics(
                        df_is,
                        strategy_daily_return_col=f'{t}_daily_pct_returns',
                        strategy_trade_count_col=f'{t}_position_count',
                        annual_trading_days=365,
                        include_transaction_costs_and_fees=False
                    )
                    for c in ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']:
                        row[f'{t}_{c}'] = tmet[c]

            is_rows.append(row)

        print(f"[WFA] Fold {fold_idx}: IS grid complete in {time.perf_counter()-t0:.1f}s. Scoring…", flush=True)

        # attach IS rows and compute scores within the fold
        is_block = pd.DataFrame(is_rows)

        # guardrail: activity
        pre_n = len(is_block)
        is_block['ok'] = pd.to_numeric(is_block['trade_count'], errors='coerce').fillna(0) >= min_trades_is
        if is_block['ok'].any():
            is_block = is_block[is_block['ok']].copy()
            print(f"[WFA] Fold {fold_idx}: guardrail kept {len(is_block)}/{pre_n} configs (min_trades_is={min_trades_is}).", flush=True)
        else:
            print(f"[WFA] Fold {fold_idx}: guardrail filtered all {pre_n} configs; continuing with all (no guardrail).", flush=True)

        # IS composite score
        is_block['is_score'] = composite_is_score(is_block)

        # pick best by score (tie-breaks implicit by sort order)
        winner = is_block.sort_values(['is_score','annualized_sharpe_ratio','max_drawdown','trade_count'],
                                      ascending=[False, False, True, True]).iloc[0]

        print(f"[WFA] Fold {fold_idx}: WINNER (IS) "
              f"tv={winner.get('annualized_target_volatility')}, "
              f"stop={winner.get('stop_loss_strategy')}, "
              f"k={winner.get('atr_multiplier')}, "
              f"atr_w={winner.get('rolling_atr_window')}, "
              f"hh={winner.get('highest_high_window')}, "
              f"cd={winner.get('cooldown_counter_threshold')} | "
              f"score={winner['is_score']:.3f}, "
              f"Sharpe={winner['annualized_sharpe_ratio']:.3f}, "
              f"MDD={winner['max_drawdown']:.3%}, "
              f"trades={int(winner['trade_count'])}", flush=True)

        # append IS rows to df_performance
        df_performance = pd.concat([df_performance, is_block[perf_cols]], ignore_index=True)

        # ---- 2) Run the promoted config out-of-sample ----
        promoted_params = {
            'annualized_target_volatility': winner.get('annualized_target_volatility', fixed_params.get('annualized_target_volatility')),
            'stop_loss_strategy':           winner.get('stop_loss_strategy', fixed_params.get('stop_loss_strategy')),
            'rolling_atr_window':           winner.get('rolling_atr_window', fixed_params.get('rolling_atr_window')),
            'atr_multiplier':               winner.get('atr_multiplier', fixed_params.get('atr_multiplier')),
            'highest_high_window':          winner.get('highest_high_window', fixed_params.get('highest_high_window')),
            'cooldown_counter_threshold':   winner.get('cooldown_counter_threshold', fixed_params.get('cooldown_counter_threshold')),
        }
        p_os = fixed_params.copy()
        p_os.update(promoted_params)
        p_os = _clean_params(p_os)

        print(f"[WFA] Fold {fold_idx}: running OS with promoted params: {_fmt_params(p_os)}", flush=True)
        t1 = time.perf_counter()

        df_os = tf_fn(
            start_date=start_os - pd.Timedelta(days=warmup_days),
            end_date=end_os, ticker_list=ticker_list,
            **p_os,
            use_specific_start_date=True,
            signal_start_date=start_os,
        )
        df_os = df_os[df_os.index >= pd.Timestamp(start_os).date()]
        df_os = perf.calculate_asset_level_returns(df_os, end_os, ticker_list)

        os_port = perf_fn(
            df_os,
            strategy_daily_return_col='portfolio_daily_pct_returns',
            strategy_trade_count_col='count_of_positions',
            include_transaction_costs_and_fees=False,
            passive_trade_rate=0.05, annual_trading_days=365,
            transaction_cost_est=0.001
        )

        print(f"[WFA] Fold {fold_idx}: OS METRICS: Sharpe={os_port.get('annualized_sharpe_ratio',np.nan):.3f}, "
              f"MDD={os_port.get('max_drawdown',np.nan):.3%}, "
              f"trades={int(os_port.get('trade_count',0))} "
              f"(elapsed {time.perf_counter()-t1:.1f}s)", flush=True)

        os_row = {
            'sampling_category':'out_sample',
            'start_date': start_os,
            'end_date': end_os,
            'is_score': float(winner['is_score']),
            'os_score': float(os_port['annualized_sharpe_ratio']),  # OS score = OS Sharpe
            'is_promoted': True,
            **{k: p_os.get(k, np.nan) for k in [
                'annualized_target_volatility','stop_loss_strategy','atr_multiplier',
                'rolling_atr_window','highest_high_window','cooldown_counter_threshold'
            ]},
            **os_port
        }
        if include_ticker_metrics:
            for t in ticker_list:
                tmet = perf.calculate_risk_and_performance_metrics(
                    df_os,
                    strategy_daily_return_col=f'{t}_daily_pct_returns',
                    strategy_trade_count_col=f'{t}_position_count',
                    annual_trading_days=365,
                    include_transaction_costs_and_fees=False
                )
                for c in ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']:
                    os_row[f'{t}_{c}'] = tmet[c]

        df_performance.loc[len(df_performance)] = os_row

        print(f"[WFA] Fold {fold_idx}: complete. Rolling forward…", flush=True)
        # ---- roll forward ----
        start_is = (start_is + (OS_LEN if step_equals_os else pd.DateOffset(months=1))).date()

    print(f"\n[WFA] finished. folds={fold_idx}, total_rows={len(df_performance)} "
          f"(elapsed {time.perf_counter()-start_ts:.1f}s)", flush=True)

    return df_performance


## Target Volatility Walk Forward Analysis

In [76]:
fixed = dict(
    # your frozen prod config passed into tf_fn
    fast_mavg=20, slow_mavg=200, mavg_stepsize=8, mavg_z_score_window=126,
    entry_rolling_donchian_window=56, exit_rolling_donchian_window=28,
    use_donchian_exit_gate=False,
    ma_crossover_signal_weight=0.9, donchian_signal_weight=0.1, weighted_signal_ewm_window=4,
    rolling_r2_window=100, lower_r_sqr_limit=0.45, upper_r_sqr_limit=0.9,
    r2_smooth_window=3, r2_confirm_days=0, r2_strong_threshold=0.75,
    log_std_window=14, coef_of_variation_window=20, vol_of_vol_z_score_window=126, vol_of_vol_p_min=0.10,
    use_activation=False, tanh_activation_constant_dict=None,
    moving_avg_type='exponential', long_only=True, price_or_returns_calc='price',
    initial_capital=15000, rolling_cov_window=20, volatility_window=30,
    stop_loss_strategy='Chandelier', rolling_atr_window=14,
    atr_multiplier=2.25, highest_high_window=56, cooldown_counter_threshold=1,
    annualized_target_volatility=0.55,  # fallback if sweep doesn't set it
    transaction_cost_est=0.001, passive_trade_rate=0.05,
    notional_threshold_pct=0.10, rolling_sharpe_window=50, cash_buffer_percentage=0.10,
    annual_trading_days=365, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-07-31',
    # use_specific_start_date=True, signal_start_date='2022-04-01'
)
sweep = {"annualized_target_volatility":[0.40,0.50,0.55,0.60,0.70]}

df_tv = run_wfa(
    start_date="2022-04-01", end_date="2025-07-31",
    ticker_list=['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD'],
    is_months=18, os_months=3, step_equals_os=True, warmup_days=300,
    fixed_params=fixed, sweep_params=sweep,
    tf_fn=tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol,
    perf_fn=calculate_risk_and_performance_metrics,
    include_ticker_metrics=True
)


[WFA] init: IS=18m, OS=3m, STEP=OS_LEN, warmup=300d, tickers=5, grid_size=5

[WFA] Fold 1: IS 2022-04-01 → 2023-09-30 | OS 2023-10-01 → 2023-12-31 | warmup=300d
[WFA] Fold 1: evaluating 5 config(s) in-sample…
[WFA] Fold 1 | IS cfg 1/5: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=2.25, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=1
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
[WFA] Fold 1 | IS cfg 1/5 METRICS: Sharpe=0.150, MDD=-13.103%, trades=58
[WFA] Fold 1 | IS cfg 2/5: annualized_target_volatility=0.5, stop_loss_strategy=Chandelier, atr_multiplier=2.25, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=1
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getti

In [87]:
df_tv

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,in_sample,2022-04-01,2023-09-30,0.062030,0.150180,0.473397,0.386539,-0.131031,292 days,0.100365,0.556783,0.577903,58.0,0.40,Chandelier,2.25,14,56,1,0.423265,NaN,False,-0.007595,-0.439854,0.266515,-0.127637,0.020862,-0.594587,0.130958,-0.036686,0.039333,-0.111741,0.646857,-0.022147,-0.030570,-0.926539,0.293654,-0.111915,0.008067,-1.666147,0.091952,-0.014681,0.033652
1,in_sample,2022-04-01,2023-09-30,0.082740,0.251422,0.608804,0.485030,-0.135905,292 days,0.100365,0.604208,0.545956,57.0,0.50,Chandelier,2.25,14,56,1,0.574715,NaN,False,-0.001794,-0.368559,0.277505,-0.130493,0.027248,-0.362048,0.156629,-0.045688,0.043134,-0.028725,0.394093,-0.029176,-0.038148,-0.803790,0.364993,-0.138083,0.012181,-1.066856,0.124882,-0.025637,0.029939
2,in_sample,2022-04-01,2023-09-30,0.089059,0.273599,0.613952,0.583024,-0.145059,292 days,0.102190,0.604660,0.545656,57.0,0.55,Chandelier,2.25,14,56,1,0.480356,NaN,False,0.006367,-0.304262,0.274612,-0.122420,0.026697,-0.336930,0.170430,-0.048243,0.046199,0.010380,0.422925,-0.033944,-0.040931,-0.694602,0.427412,-0.156670,0.013448,-0.934778,0.137117,-0.028189,0.060043
3,in_sample,2022-04-01,2023-09-30,0.098075,0.307117,0.651815,0.623411,-0.150465,292 days,0.100365,0.635967,0.525064,59.0,0.60,Chandelier,2.25,14,56,1,0.220550,NaN,False,0.002035,-0.330487,0.281130,-0.124702,0.028099,-0.326153,0.165298,-0.039641,0.050368,0.053953,0.460905,-0.036930,-0.036057,-0.626262,0.442514,-0.161408,0.014388,-0.849444,0.146444,-0.030689,0.039018
4,in_sample,2022-04-01,2023-09-30,0.107947,0.337019,0.655483,0.660664,-0.164682,292 days,0.104015,0.652556,0.514317,63.0,0.70,Chandelier,2.25,14,56,1,-0.379549,NaN,False,0.013310,-0.235396,0.282342,-0.124732,0.026165,-0.360799,0.163055,-0.039586,0.059086,0.126599,0.523094,-0.042058,-0.051541,-0.651922,0.500961,-0.178276,0.022767,-0.598874,0.154056,-0.030714,0.056194
5,out_sample,2023-10-01,2023-12-31,8.559929,5.051270,123.856993,0.633967,-0.069111,22 days,0.500000,2.589370,0.011195,47.0,0.50,Chandelier,2.25,14,56,1,0.574715,5.051270,True,0.197930,0.805460,0.261972,-0.047620,-0.000114,-0.919299,0.093311,-0.022302,0.241325,1.392554,0.193601,-0.030912,1.585598,5.401248,0.192072,-0.027257,2.333130,4.255250,0.449820,-0.022730,0.267934
6,in_sample,2022-07-01,2023-12-31,0.515139,1.761367,5.204459,0.428444,-0.098980,153 days,0.178506,2.429623,0.015435,94.0,0.40,Chandelier,2.25,14,56,1,2.488981,NaN,False,0.019613,-0.155371,0.256914,-0.127798,0.019765,-0.581772,0.118209,-0.044809,0.070900,0.276397,0.345170,-0.044577,0.156967,1.083642,0.207667,-0.042807,0.194538,1.328109,0.305386,-0.016654,0.071111
7,in_sample,2022-07-01,2023-12-31,0.614936,1.684665,4.618796,0.560074,-0.133138,159 days,0.176685,2.281161,0.022922,108.0,0.50,Chandelier,2.25,14,56,1,1.686007,NaN,False,0.027858,-0.081462,0.272251,-0.130806,0.033380,-0.230939,0.138535,-0.045851,0.069797,0.228252,0.286334,-0.054929,0.185414,1.098212,0.259315,-0.057876,0.231545,1.331651,0.336869,-0.025111,0.120147
8,in_sample,2022-07-01,2023-12-31,0.671074,1.734672,4.973026,0.588984,-0.134943,159 days,0.178506,2.332251,0.020049,109.0,0.55,Chandelier,2.25,14,56,1,1

In [89]:
df_tv['vol_tracking_error'] = (np.abs(df_tv['annualized_std_dev'] - df_tv['annualized_target_volatility']) / df_tv['annualized_target_volatility'])

in_sample_cond = (df_tv.sampling_category == 'in_sample')
df_tv_is = df_tv[in_sample_cond].reset_index(drop=True)
df_tv_os = df_tv[~in_sample_cond].reset_index(drop=True)

In [91]:
df_tv.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Target_Volatility_Performance-2022-04-01-2025-10-01.pickle')

In [95]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'annualized_std_dev':['median','mean','std'],
            'vol_tracking_error':['median','mean','std'],
            'trade_count':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}

In [97]:
df_tv_is.groupby(['annualized_target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio                      \
                                              median      mean       std   
annualized_target_volatility                                               
0.40                                        1.761367  1.609636  0.686733   
0.55                                        1.734672  1.531630  0.587578   
0.50                                        1.684665  1.522892  0.601438   
0.60                                        1.674058  1.510240  0.570507   
0.70                                        1.631542  1.502700  0.559738   

                             annualized_return                      \
                                        median      mean       std   
annualized_target_volatility                                         
0.40                                  0.515139  0.513264  0.232470   
0.55                                  0.671074  0.618087  0.271165   
0.50                                  0.613093  0.580513  0.263852   
0.60                                  0.671616  0.628047  0.276296   
0.70                                  0.680846  0.656702  0.292981   

                             max_drawdown                      \
                                   median      mean       std   
annualized_target_volatility                                    
0.40                            -0.175480 -0.169407  0.049690   
0.55                            -0.227879 -0.212002  0.058041   
0.50                            -0.213488 -0.200800  0.055260   
0.60                            -0.237117 -0.221300  0.059913   
0.70                            -0.242743 -0.233133  0.059493   

                             annualized_std_dev                      \
                                         median      mean       std   
annualized_target_volatility                                          
0.40                                   0.428444  0.433328  0.029262   
0.55                                   0.583024  0.564907  0.032707   
0.50                                   0.493990  0.519426  0.037084   
0.60                                   0.612973  0.587195  0.042597   
0.70                                   0.636773  0.604931  0.066728   

                             vol_tracking_error                      \
                                         median      mean       std   
annualized_target_volatility                                          
0.40                                   0.071111  0.092936  0.058090   
0.55                                   0.060043  0.056659  0.025459   
0.50                                   0.029939  0.061661  0.053161   
0.60                                   0.048835  0.061054  0.034970   
0.70                                   0.090325  0.135813  0.095326   

                             trade_count                         \
                                  median        mean        std   
annualized_target_volatility                                      
0.40                               104.0   97.428571  20.525246   
0.55                               122.0  113.428571  27.748016   
0.50                               116.0  108.857143  25.281275   
0.60                               128.0  117.571429  28.970428   
0.70                               132.0  121.714286  28.848537   

                             BTC-USD_annualized_sharpe_ratio            \
                                                      median      mean   
annualized_target_volatility                                             
0.40                                               -0.155371  0.059218   
0.55                                               -0.039606  0.096297   
0.50                                               -0.081462  0.074563   
0.60                                               -0.035473  0.055127   
0.70                                                0.047208  0.097584   

                                       ETH-USD_annualized_sharpe_ratio

In [99]:
df_tv_os.groupby(['annualized_target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio                     \
                                              median      mean      std   
annualized_target_volatility                                              
0.5                                         5.051270  5.051270      NaN   
0.4                                        -0.836667 -1.175719  2.86716   

                             annualized_return                      \
                                        median      mean       std   
annualized_target_volatility                                         
0.5                                   8.559929  8.559929       NaN   
0.4                                  -0.079071  0.067714  0.518424   

                             max_drawdown                      \
                                   median      mean       std   
annualized_target_volatility                                    
0.5                             -0.069111 -0.069111       NaN   
0.4                             -0.095452 -0.091789  0.043373   

                             annualized_std_dev                      \
                                         median      mean       std   
annualized_target_volatility                                          
0.5                                    0.633967  0.633967       NaN   
0.4                                    0.300797  0.301143  0.117193   

                             vol_tracking_error                      \
                                         median      mean       std   
annualized_target_volatility                                          
0.5                                    0.267934  0.267934       NaN   
0.4                                    0.248007  0.291520  0.239067   

                             trade_count                        \
                                  median       mean        std   
annualized_target_volatility                                     
0.5                                 47.0  47.000000        NaN   
0.4                                 16.0  16.666667  10.033278   

                             BTC-USD_annualized_sharpe_ratio               \
                                                      median     mean std   
annualized_target_volatility                                                
0.5                                                  0.80546  0.80546 NaN   
0.4                                                 -1.81793     -inf NaN   

                             ETH-USD_annualized_sharpe_ratio                \
                                                      median      mean std   
annualized_target_volatility                                                 
0.5                                                -0.919299 -0.919299 NaN   
0.4                                                     -inf      -inf NaN   

                             SOL-USD_annualized_sharpe_ratio                \
                                                      median      mean std   
annualized_target_volatility                                                 
0.5                                                 1.392554  1.392554 NaN   
0.4                                                     -inf      -inf NaN   

                             ADA-USD_annualized_sharpe_ratio                \
                                                      median      mean std   
annualized_target_volatility                                                 
0.5                                                 5.401248  5.401248 NaN   
0.4                                                     -inf      -inf NaN   

                             AVAX-USD_annualized_sharpe_ratio               
                                                       median     mean std  
annualized_target_volatility                                                
0.5                                                   4.25525  4.25525 NaN  
0.4                                                      -inf     

In [101]:
df_tv_os

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,out_sample,2023-10-01,2023-12-31,8.559929,5.051270,123.856993,0.633967,-0.069111,22 days,0.500000,2.589370,0.011195,47.0,0.5,Chandelier,2.25,14,56,1,0.574715,5.051270,True,0.197930,0.805460,0.261972,-0.047620,-0.000114,-0.919299,0.093311,-0.022302,0.241325,1.392554,0.193601,-0.030912,1.585598,5.401248,0.192072,-0.027257,2.333130,4.255250,0.449820,-0.022730,0.267934
1,out_sample,2024-01-01,2024-03-31,1.008826,2.420412,6.985011,0.424984,-0.144427,37 days,0.373626,1.294133,0.198931,23.0,0.4,Chandelier,2.25,14,56,1,2.488981,2.420412,True,0.083092,0.336052,0.140872,-0.058698,0.936115,2.754706,0.299632,-0.109713,-0.026717,-7.974690,0.005347,-0.006729,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.062459
2,out_sample,2024-04-01,2024-06-30,-0.243350,-3.485933,-3.300537,0.136353,-0.073730,90 days,0.054945,-1.477854,0.142938,9.0,0.4,Chandelier,2.25,14,56,1,2.899567,-3.485933,True,-0.214787,-2.912832,0.246354,-0.058503,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.659119
3,out_sample,2024-07-01,2024-09-30,0.085207,0.373578,3.914356,0.428271,-0.021768,85 days,0.043478,0.427849,0.669772,4.0,0.4,Chandelier,2.25,14,56,1,2.781900,0.373578,True,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,-0.006944,-19.400678,0.012685,-0.001889,0.134260,0.799679,0.362713,-0.022173,0.070676
4,out_sample,2024-10-01,2024-12-31,0.227801,0.821012,2.909473,0.337605,-0.078296,38 days,0.304348,0.523965,0.601577,32.0,0.4,Chandelier,2.25,14,56,1,2.466243,0.821012,True,0.470239,2.080154,0.240368,-0.034100,-0.047829,-3.438207,0.076056,-0.016832,0.054504,0.091746,0.188210,-0.042809,0.000000,-inf,NaN,0.000000,-0.041881,-1.600203,0.177761,-0.024942,0.155986
5,out_sample,2025-01-01,2025-03-31,-0.404289,-5.136470,-3.371731,0.215660,-0.119905,89 days,0.066667,-2.328531,0.022150,14.0,0.4,Chandelier,2.25,14,56,1,2.506616,-5.136470,True,-0.333097,-4.104783,0.225123,-0.095063,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,-0.022585,-6.327392,NaN,-0.005617,0.000000,-inf,NaN,0.000000,0.460850
6,out_sample,2025-04-01,2025-06-30,-0.267912,-2.046912,-2.379160,0.263989,-0.112608,47 days,0.263736,-0.878075,0.382241,18.0,0.4,Chandelier,2.25,14,56,1,2.175352,-2.046912,True,-0.035857,-0.723028,0.252532,-0.048488,0.028990,-0.274681,0.130054,-0.021096,0.051722,0.057077,0.114033,-0.014103,0.000000,-inf,NaN,0.000000,0.000000,-inf,NaN,0.000000,0.340028


In [103]:
df_tv_is

,sampling_category,start_date,end_date,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,annualized_target_volatility,stop_loss_strategy,atr_multiplier,rolling_atr_window,highest_high_window,cooldown_counter_threshold,is_score,os_score,is_promoted,BTC-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_annualized_return,ETH-USD_annualized_sharpe_ratio,ETH-USD_annualized_std_dev,ETH-USD_max_drawdown,SOL-USD_annualized_return,SOL-USD_annualized_sharpe_ratio,SOL-USD_annualized_std_dev,SOL-USD_max_drawdown,ADA-USD_annualized_return,ADA-USD_annualized_sharpe_ratio,ADA-USD_annualized_std_dev,ADA-USD_max_drawdown,AVAX-USD_annualized_return,AVAX-USD_annualized_sharpe_ratio,AVAX-USD_annualized_std_dev,AVAX-USD_max_drawdown,vol_tracking_error
0,in_sample,2022-04-01,2023-09-30,0.062030,0.150180,0.473397,0.386539,-0.131031,292 days,0.100365,0.556783,0.577903,58.0,0.40,Chandelier,2.25,14,56,1,0.423265,NaN,False,-0.007595,-0.439854,0.266515,-0.127637,0.020862,-0.594587,0.130958,-0.036686,0.039333,-0.111741,0.646857,-0.022147,-0.030570,-0.926539,0.293654,-0.111915,0.008067,-1.666147,0.091952,-0.014681,0.033652
1,in_sample,2022-04-01,2023-09-30,0.082740,0.251422,0.608804,0.485030,-0.135905,292 days,0.100365,0.604208,0.545956,57.0,0.50,Chandelier,2.25,14,56,1,0.574715,NaN,False,-0.001794,-0.368559,0.277505,-0.130493,0.027248,-0.362048,0.156629,-0.045688,0.043134,-0.028725,0.394093,-0.029176,-0.038148,-0.803790,0.364993,-0.138083,0.012181,-1.066856,0.124882,-0.025637,0.029939
2,in_sample,2022-04-01,2023-09-30,0.089059,0.273599,0.613952,0.583024,-0.145059,292 days,0.102190,0.604660,0.545656,57.0,0.55,Chandelier,2.25,14,56,1,0.480356,NaN,False,0.006367,-0.304262,0.274612,-0.122420,0.026697,-0.336930,0.170430,-0.048243,0.046199,0.010380,0.422925,-0.033944,-0.040931,-0.694602,0.427412,-0.156670,0.013448,-0.934778,0.137117,-0.028189,0.060043
3,in_sample,2022-04-01,2023-09-30,0.098075,0.307117,0.651815,0.623411,-0.150465,292 days,0.100365,0.635967,0.525064,59.0,0.60,Chandelier,2.25,14,56,1,0.220550,NaN,False,0.002035,-0.330487,0.281130,-0.124702,0.028099,-0.326153,0.165298,-0.039641,0.050368,0.053953,0.460905,-0.036930,-0.036057,-0.626262,0.442514,-0.161408,0.014388,-0.849444,0.146444,-0.030689,0.039018
4,in_sample,2022-04-01,2023-09-30,0.107947,0.337019,0.655483,0.660664,-0.164682,292 days,0.104015,0.652556,0.514317,63.0,0.70,Chandelier,2.25,14,56,1,-0.379549,NaN,False,0.013310,-0.235396,0.282342,-0.124732,0.026165,-0.360799,0.163055,-0.039586,0.059086,0.126599,0.523094,-0.042058,-0.051541,-0.651922,0.500961,-0.178276,0.022767,-0.598874,0.154056,-0.030714,0.056194
5,in_sample,2022-07-01,2023-12-31,0.515139,1.761367,5.204459,0.428444,-0.098980,153 days,0.178506,2.429623,0.015435,94.0,0.40,Chandelier,2.25,14,56,1,2.488981,NaN,False,0.019613,-0.155371,0.256914,-0.127798,0.019765,-0.581772,0.118209,-0.044809,0.070900,0.276397,0.345170,-0.044577,0.156967,1.083642,0.207667,-0.042807,0.194538,1.328109,0.305386,-0.016654,0.071111
6,in_sample,2022-07-01,2023-12-31,0.614936,1.684665,4.618796,0.560074,-0.133138,159 days,0.176685,2.281161,0.022922,108.0,0.50,Chandelier,2.25,14,56,1,1.686007,NaN,False,0.027858,-0.081462,0.272251,-0.130806,0.033380,-0.230939,0.138535,-0.045851,0.069797,0.228252,0.286334,-0.054929,0.185414,1.098212,0.259315,-0.057876,0.231545,1.331651,0.336869,-0.025111,0.120147
7,in_sample,2022-07-01,2023-12-31,0.671074,1.734672,4.973026,0.588984,-0.134943,159 days,0.178506,2.332251,0.020049,109.0,0.55,Chandelier,2.25,14,56,1,1.688894,NaN,False,0.033853,-0.039606,0.273032,-0.125210,0.036170,-0.167746,0.149236,-0.048414,0.077300,0.279836,0.298430,-0.060002,0.205064,1.182378,0.272855,-0.055236,0.241804,1.344338,0.351631,-0.027465,0.070880
8,in_sample,2022-07-01,2023-12-31,0.711719,1.748907,4.943141,0.629301,-0.143981,159 days,0.174863,2.340624,0.019609,112.0,0.60,Chandelier,2.25,14,56,1,1.5346

## Cooldown Counter Threshold

In [107]:
fixed = dict(
    # your frozen prod config passed into tf_fn
    fast_mavg=20, slow_mavg=200, mavg_stepsize=8, mavg_z_score_window=126,
    entry_rolling_donchian_window=56, exit_rolling_donchian_window=28,
    use_donchian_exit_gate=False,
    ma_crossover_signal_weight=0.9, donchian_signal_weight=0.1, weighted_signal_ewm_window=4,
    rolling_r2_window=100, lower_r_sqr_limit=0.45, upper_r_sqr_limit=0.9,
    r2_smooth_window=3, r2_confirm_days=0, r2_strong_threshold=0.75,
    log_std_window=14, coef_of_variation_window=20, vol_of_vol_z_score_window=126, vol_of_vol_p_min=0.10,
    use_activation=False, tanh_activation_constant_dict=None,
    moving_avg_type='exponential', long_only=True, price_or_returns_calc='price',
    initial_capital=15000, rolling_cov_window=20, volatility_window=30,
    stop_loss_strategy='Chandelier', rolling_atr_window=14,
    atr_multiplier=2.25, highest_high_window=56, cooldown_counter_threshold=1,
    annualized_target_volatility=0.40,  # fallback if sweep doesn't set it
    transaction_cost_est=0.001, passive_trade_rate=0.05,
    notional_threshold_pct=0.10, rolling_sharpe_window=50, cash_buffer_percentage=0.10,
    annual_trading_days=365, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-07-31',
    # use_specific_start_date=True, signal_start_date='2022-04-01'
)
sweep = {"cooldown_counter_threshold": [0, 1, 2, 3, 5]}

df_cooldown = run_wfa(
    start_date="2022-04-01", end_date="2025-07-31",
    ticker_list=['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD'],
    is_months=18, os_months=3, step_equals_os=True, warmup_days=300,
    fixed_params=fixed, sweep_params=sweep,
    tf_fn=tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol,
    perf_fn=calculate_risk_and_performance_metrics,
    include_ticker_metrics=True
)


[WFA] init: IS=18m, OS=3m, STEP=OS_LEN, warmup=300d, tickers=5, grid_size=5

[WFA] Fold 1: IS 2022-04-01 → 2023-09-30 | OS 2023-10-01 → 2023-12-31 | warmup=300d
[WFA] Fold 1: evaluating 5 config(s) in-sample…
[WFA] Fold 1 | IS cfg 1/5: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=2.25, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=0
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
[WFA] Fold 1 | IS cfg 1/5 METRICS: Sharpe=0.150, MDD=-13.103%, trades=58
[WFA] Fold 1 | IS cfg 2/5: annualized_target_volatility=0.4, stop_loss_strategy=Chandelier, atr_multiplier=2.25, rolling_atr_window=14, highest_high_window=56, cooldown_counter_threshold=1
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getti

In [109]:
df_cooldown['vol_tracking_error'] = (np.abs(df_cooldown['annualized_std_dev'] - df_cooldown['annualized_target_volatility']) / df_cooldown['annualized_target_volatility'])

in_sample_cond = (df_cooldown.sampling_category == 'in_sample')
df_cooldown_is = df_cooldown[in_sample_cond].reset_index(drop=True)
df_cooldown_os = df_cooldown[~in_sample_cond].reset_index(drop=True)

In [111]:
df_cooldown.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/research/backtest_results/trend_following_results/Cooldown_Counter_Threshold_Performance-2022-04-01-2025-10-01.pickle')